In [1]:
import re
import copy
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from langchain.schema import Document
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import Pinecone

In [2]:
# process .env file
load_dotenv()

True

In [4]:
# Access the environment variables
openai_api_key = os.getenv('FINE_TUNED_TOXIC_DETECTION_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc_index = os.getenv('PINECONE_GPT')

In [5]:
embeddings = OpenAIEmbeddings()

In [10]:
documents = []
df = pd.read_csv('./data/csv/unintended_bias_toxicity_classification_set.csv')
df.head()

,Text,Toxic
0,idiot holding sign saying hate Blacks going wi...,0
1,Stay safe folks rain year sudden unpredictable...,0
2,basically gave Canadian reward murdering Ameri...,0
3,would lose job cave political masters doubt es...,0
4,Yesshame,1


In [11]:
for index, row in df.iterrows():

    # Example of creating a document with text and metadata
    documents.append(Document(
        page_content=f"{index + 1}. {row['Text']} - Label: {row['Toxic']}"
    ))

In [12]:
len(documents)

1000

In [13]:
# insert splits into Pinecone vector database as embeddings
PineconeVectorStore.from_documents(documents, embeddings, index_name=pc_index)